# Embedding Wikipedia articles for search

This notebook shows how we prepared a dataset of Wikipedia articles for search, used in 3_Question_answering_using_embeddings.ipynb

Procedure:

0. Prerequisites: Import libraries, set API key (if needed)
1. Collect: We download a few hundred Wikipedia articles about the 2022 Olympics
2. Chunk: Documents are split into short, semi-self-contained sections to be embedded
3. Embed: Each section is embedded with the OpenAI API
4. Store: Embeddings are saved in a CSV file (for large datasets, use a vector database)

## 0. Prerequisites

### Import libraries

In [2]:
!pip install openai tiktoken pandas mwclient mwparserfromhell regex

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 29.0 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [20]:
# imports
import mwclient  # for downloading example Wikipedia articles
import mwparserfromhell  # for splitting Wikipedia articles into sections
import openai  # for generating embeddings
import pandas as pd  # for DataFrames to store article sections and embeddings
import re  # for cutting <ref> links out of Wikipedia articles
import tiktoken  # for counting tokens
from dotenv import load_dotenv, find_dotenv
import os
import time

## 1. Collect documents

In this example, we'll download a few hundred Wikipedia articles related to the 2022 Winter Olympics.

In [6]:
# Load OpenAI API Key from credentials.env
load_dotenv(find_dotenv("credentials.env"), override=True)
openai.api_key = os.environ["OPENAI_API_KEY"]

In [7]:
# get Wikipedia pages about the 2022 Winter Olympics

CATEGORY_TITLE = "Category:2022 Winter Olympics"
WIKI_SITE = "en.wikipedia.org"


def titles_from_category(
    category: mwclient.listing.Category, max_depth: int
) -> set[str]:
    """Return a set of page titles in a given Wiki category and its subcategories."""
    titles = set()
    for cm in category.members():
        if type(cm) == mwclient.page.Page:
            # ^type() used instead of isinstance() to catch match w/ no inheritance
            titles.add(cm.name)
        elif isinstance(cm, mwclient.listing.Category) and max_depth > 0:
            deeper_titles = titles_from_category(cm, max_depth=max_depth - 1)
            titles.update(deeper_titles)
    return titles


site = mwclient.Site(WIKI_SITE)
category_page = site.pages[CATEGORY_TITLE]
titles = titles_from_category(category_page, max_depth=1)
# ^note: max_depth=1 means we go one level deep in the category tree
print(f"Found {len(titles)} article titles in {CATEGORY_TITLE}.")

Found 732 article titles in Category:2022 Winter Olympics.


## 2. Chunk documents

Now that we have our reference documents, we need to prepare them for search.

Because GPT can only read a limited amount of text at once, we'll split each document into chunks short enough to be read.

For this specific example on Wikipedia articles, we'll:
- Discard less relevant-looking sections like External Links and Footnotes
- Clean up the text by removing reference tags (e.g., <ref>), whitespace, and super short sections
- Split each article into sections
- Prepend titles and subtitles to each section's text, to help GPT understand the context
- If a section is long (say, > 1,600 tokens), we'll recursively split it into smaller sections, trying to split along semantic boundaries like paragraphs

In [8]:
# define functions to split Wikipedia pages into sections

SECTIONS_TO_IGNORE = [
    "See also",
    "References",
    "External links",
    "Further reading",
    "Footnotes",
    "Bibliography",
    "Sources",
    "Citations",
    "Literature",
    "Footnotes",
    "Notes and references",
    "Photo gallery",
    "Works cited",
    "Photos",
    "Gallery",
    "Notes",
    "References and sources",
    "References and notes",
]


def all_subsections_from_section(
    section: mwparserfromhell.wikicode.Wikicode,
    parent_titles: list[str],
    sections_to_ignore: set[str],
) -> list[tuple[list[str], str]]:
    """
    From a Wikipedia section, return a flattened list of all nested subsections.
    Each subsection is a tuple, where:
        - the first element is a list of parent subtitles, starting with the page title
        - the second element is the text of the subsection (but not any children)
    """
    headings = [str(h) for h in section.filter_headings()]
    title = headings[0]
    if title.strip("=" + " ") in sections_to_ignore:
        # ^wiki headings are wrapped like "== Heading =="
        return []
    titles = parent_titles + [title]
    full_text = str(section)
    section_text = full_text.split(title)[1]
    if len(headings) == 1:
        return [(titles, section_text)]
    else:
        first_subtitle = headings[1]
        section_text = section_text.split(first_subtitle)[0]
        results = [(titles, section_text)]
        for subsection in section.get_sections(levels=[len(titles) + 1]):
            results.extend(all_subsections_from_section(subsection, titles, sections_to_ignore))
        return results


def all_subsections_from_title(
    title: str,
    sections_to_ignore: set[str] = SECTIONS_TO_IGNORE,
    site_name: str = WIKI_SITE,
) -> list[tuple[list[str], str]]:
    """From a Wikipedia page title, return a flattened list of all nested subsections.
    Each subsection is a tuple, where:
        - the first element is a list of parent subtitles, starting with the page title
        - the second element is the text of the subsection (but not any children)
    """
    site = mwclient.Site(site_name)
    page = site.pages[title]
    text = page.text()
    parsed_text = mwparserfromhell.parse(text)
    headings = [str(h) for h in parsed_text.filter_headings()]
    if headings:
        summary_text = str(parsed_text).split(headings[0])[0]
    else:
        summary_text = str(parsed_text)
    results = [([title], summary_text)]
    for subsection in parsed_text.get_sections(levels=[2]):
        results.extend(all_subsections_from_section(subsection, [title], sections_to_ignore))
    return results


In [9]:
# split pages into sections
# may take ~1 minute per 100 articles
wikipedia_sections = []
for title in titles:
    wikipedia_sections.extend(all_subsections_from_title(title))
print(f"Found {len(wikipedia_sections)} sections in {len(titles)} pages.")


Found 5742 sections in 732 pages.


In [12]:
# clean text
def clean_section(section: tuple[list[str], str]) -> tuple[list[str], str]:
    """
    Return a cleaned up section with:
        - <ref>xyz</ref> patterns removed
        - leading/trailing whitespace removed
    """
    titles, text = section
    text = re.sub(r"<ref.*?</ref>", "", text)
    text = text.strip()
    return (titles, text)


wikipedia_sections = [clean_section(ws) for ws in wikipedia_sections]

# filter out short/blank sections
def keep_section(section: tuple[list[str], str]) -> bool:
    """Return True if the section should be kept, False otherwise."""
    titles, text = section
    if len(text) < 16:
        return False
    else:
        return True


original_num_sections = len(wikipedia_sections)
wikipedia_sections = [ws for ws in wikipedia_sections if keep_section(ws)]
print(f"Filtered out {original_num_sections-len(wikipedia_sections)} sections, leaving {len(wikipedia_sections)} sections.")


Filtered out 529 sections, leaving 5213 sections.


In [23]:
# print example data
for ws in wikipedia_sections[:5]:
    print(ws[0])
    display(ws[1][:500] + "...")
    print()


['Michela Moioli']


'{{short description|Italian snowboarder}}\n{{Use dmy dates|date=February 2021}}\n{{Infobox snowboarder\n| name             = Michela Moioli\n| image            = Michela Moioli (ITA).jpg\n| image_size       = \n| birth_date       = {{birth date and age|1995|7|17|df=y}}\n| birth_place      = [[Alzano Lombardo]], [[Italy]]\n| nationality      = Italian\n| height           = 1.73 m\n| weight           = 65 kg\n| club             = [[Centro Sportivo Esercito|C.S. Esercito]]\n| coach            = Luca Pozzolini\n...'


['Michela Moioli', '==Biography==']


'Qualified young at the age of 18 for the [[Winter Olympics|Olympic]] final of the [[snowboardcross]] in [[Sochi 2014]], a bad fall ousted her from the fight for the podium. As a result of this injury, she had to undergo a [[cruciate ligament]] surgery that caused her to close the [[Snowboarding World Cup|World Cup]] competitive season prematurely....'


['Michela Moioli', '==World Cup==', '===Overall===']


'{| class="wikitable" style="text-align:center"\n|- class="hintergrundfarbe5"\n! style="width:60px;"| Season !! style="width:40px;"| Rank !! style="width:60px;"| Points\n|-\n| [[2012–13 FIS Snowboard World Cup|2013]] || style="background:#c96;"|3rd || 3170\n|-\n| [[2013–14 FIS Snowboard World Cup|2014]] || 27th || 540\n|-\n| [[2014–15 FIS Snowboard World Cup|2015]] || style="background:#c96;"|3rd || 1900\n|-\n| [[2015–16 FIS Snowboard World Cup|2016]] ||bgcolor=gold|1st || 5100\n|-\n| [[2016–17 FIS Snowboard...'


['Michela Moioli', '==World Cup==', '===Podium===']


'{| class="wikitable sortable"  style="width:50%; text-align:center;"\n|-\n!#!!Date!!Place!!Discipline!!Rank\n|-\n|1 ||17 February 2013|| align="left" |{{flagicon|RUS}} [[Sochi]]|| Snowboardcross || bgcolor="gold" |1st\n|-\n|2 ||9 March 2013|| align="left" |{{flagicon|SUI}} [[Arosa]]|| Snowboardcross || style="background:silver;" |2nd\n|-\n|3 ||14 March 2015|| align="left" |{{flagicon|SUI}} [[Veysonnaz]]|| Snowboardcross || bgcolor="gold" |1st\n|-\n|4 ||23 January 2016|| align="left" |{{flagicon|GER}} [[Fe...'


['Lviv bid for the 2022 Winter Olympics']


"{{Olympic bid|2022|Winter|\n| Paralympics = yes\n| logo = Lviv 2022 Winter Olympics bid.svg\n| logo-size = 220px\n| fullname = [[Lviv]], [[Ukraine]]\n| chair = \n| committee = [[National Olympic Committee of Ukraine]] (UKR)\n| history = None\n}}\n\n'''Lviv 2022''' ({{lang-uk|Львів 2022}}; {{lang-pl|Lwów 2022}}; {{lang-ru|Львов 2022}}; {{lang-de|Lemberg 2022}}) was a bid by the city of [[Lviv]] and the [[National Olympic Committee of Ukraine]] for the [[2022 Winter Olympics]]. Lviv withdrew its bid on 30 J..."

Next, we'll recursively split long sections into smaller sections. There's no perfect recipe for splitting text into sections. Some tradeoffs include:
- Longer sections may be better for questions that require more context
- Longer sections may be worse for retrieval, as they may have more topics muddled together
- Shorter sections are better for reducing costs (which are proportional to the number of tokens)
- Shorter sections allow more sections to be retrieved, which may help with recall
- Overlapping sections may help prevent answers from being cut by section boundaries

Here, we'll use a simple approach and limit sections to 1,600 tokens each, recursively halving any sections that are too long. To avoid cutting in the middle of useful sentences, we'll split along paragraph boundaries when possible.

In [14]:
GPT_MODEL = "gpt-3.5-turbo"  # only matters insofar as it selects which tokenizer to use


def num_tokens(text: str, model: str = GPT_MODEL) -> int:
    """Return the number of tokens in a string."""
    encoding = tiktoken.encoding_for_model(model)
    return len(encoding.encode(text))


def halved_by_delimiter(string: str, delimiter: str = "\n") -> list[str, str]:
    """Split a string in two, on a delimiter, trying to balance tokens on each side."""
    chunks = string.split(delimiter)
    if len(chunks) == 1:
        return [string, ""]  # no delimiter found
    elif len(chunks) == 2:
        return chunks  # no need to search for halfway point
    else:
        total_tokens = num_tokens(string)
        halfway = total_tokens // 2
        best_diff = halfway
        for i, chunk in enumerate(chunks):
            left = delimiter.join(chunks[: i + 1])
            left_tokens = num_tokens(left)
            diff = abs(halfway - left_tokens)
            if diff >= best_diff:
                break
            else:
                best_diff = diff
        left = delimiter.join(chunks[:i])
        right = delimiter.join(chunks[i:])
        return [left, right]


def truncated_string(
    string: str,
    model: str,
    max_tokens: int,
    print_warning: bool = True,
) -> str:
    """Truncate a string to a maximum number of tokens."""
    encoding = tiktoken.encoding_for_model(model)
    encoded_string = encoding.encode(string)
    truncated_string = encoding.decode(encoded_string[:max_tokens])
    if print_warning and len(encoded_string) > max_tokens:
        print(f"Warning: Truncated string from {len(encoded_string)} tokens to {max_tokens} tokens.")
    return truncated_string


def split_strings_from_subsection(
    subsection: tuple[list[str], str],
    max_tokens: int = 1000,
    model: str = GPT_MODEL,
    max_recursion: int = 5,
) -> list[str]:
    """
    Split a subsection into a list of subsections, each with no more than max_tokens.
    Each subsection is a tuple of parent titles [H1, H2, ...] and text (str).
    """
    titles, text = subsection
    string = "\n\n".join(titles + [text])
    num_tokens_in_string = num_tokens(string)
    # if length is fine, return string
    if num_tokens_in_string <= max_tokens:
        return [string]
    # if recursion hasn't found a split after X iterations, just truncate
    elif max_recursion == 0:
        return [truncated_string(string, model=model, max_tokens=max_tokens)]
    # otherwise, split in half and recurse
    else:
        titles, text = subsection
        for delimiter in ["\n\n", "\n", ". "]:
            left, right = halved_by_delimiter(text, delimiter=delimiter)
            if left == "" or right == "":
                # if either half is empty, retry with a more fine-grained delimiter
                continue
            else:
                # recurse on each half
                results = []
                for half in [left, right]:
                    half_subsection = (titles, half)
                    half_strings = split_strings_from_subsection(
                        half_subsection,
                        max_tokens=max_tokens,
                        model=model,
                        max_recursion=max_recursion - 1,
                    )
                    results.extend(half_strings)
                return results
    # otherwise no split was found, so just truncate (should be very rare)
    return [truncated_string(string, model=model, max_tokens=max_tokens)]


In [15]:
# split sections into chunks
MAX_TOKENS = 1600
wikipedia_strings = []
for section in wikipedia_sections:
    wikipedia_strings.extend(split_strings_from_subsection(section, max_tokens=MAX_TOKENS))

print(f"{len(wikipedia_sections)} Wikipedia sections split into {len(wikipedia_strings)} strings.")


5213 Wikipedia sections split into 6078 strings.


In [16]:
# print example data
print(wikipedia_strings[1])


Michela Moioli

==Biography==

Qualified young at the age of 18 for the [[Winter Olympics|Olympic]] final of the [[snowboardcross]] in [[Sochi 2014]], a bad fall ousted her from the fight for the podium. As a result of this injury, she had to undergo a [[cruciate ligament]] surgery that caused her to close the [[Snowboarding World Cup|World Cup]] competitive season prematurely.


## 3. Embed document chunks

Now that we've split our library into shorter self-contained strings, we can compute embeddings for each. For large embedding jobs, use a script to parallelize requests while throttling to stay under rate limits.

In [21]:
# calculate embeddings
EMBEDDING_MODEL = "text-embedding-ada-002"  # OpenAI's best embeddings as of Apr 2023
BATCH_SIZE = 100  # you can submit up to 2048 embedding inputs per request

embeddings = []
for batch_start in range(0, len(wikipedia_strings), BATCH_SIZE):
    batch_end = batch_start + BATCH_SIZE
    batch = wikipedia_strings[batch_start:batch_end]
    print(f"Batch {batch_start} to {batch_end-1}")
    response = openai.Embedding.create(model=EMBEDDING_MODEL, input=batch)
    for i, be in enumerate(response["data"]):
        assert i == be["index"]  # double check embeddings are in same order as input
    batch_embeddings = [e["embedding"] for e in response["data"]]
    embeddings.extend(batch_embeddings)
    time.sleep(5)

df = pd.DataFrame({"text": wikipedia_strings, "embedding": embeddings})


Batch 0 to 99
Batch 100 to 199
Batch 200 to 299
Batch 300 to 399
Batch 400 to 499
Batch 500 to 599
Batch 600 to 699
Batch 700 to 799
Batch 800 to 899
Batch 900 to 999
Batch 1000 to 1099
Batch 1100 to 1199
Batch 1200 to 1299
Batch 1300 to 1399
Batch 1400 to 1499
Batch 1500 to 1599
Batch 1600 to 1699
Batch 1700 to 1799
Batch 1800 to 1899
Batch 1900 to 1999
Batch 2000 to 2099
Batch 2100 to 2199
Batch 2200 to 2299
Batch 2300 to 2399
Batch 2400 to 2499
Batch 2500 to 2599
Batch 2600 to 2699
Batch 2700 to 2799
Batch 2800 to 2899
Batch 2900 to 2999
Batch 3000 to 3099
Batch 3100 to 3199
Batch 3200 to 3299
Batch 3300 to 3399
Batch 3400 to 3499
Batch 3500 to 3599
Batch 3600 to 3699
Batch 3700 to 3799
Batch 3800 to 3899
Batch 3900 to 3999
Batch 4000 to 4099
Batch 4100 to 4199
Batch 4200 to 4299
Batch 4300 to 4399
Batch 4400 to 4499
Batch 4500 to 4599
Batch 4600 to 4699
Batch 4700 to 4799
Batch 4800 to 4899
Batch 4900 to 4999
Batch 5000 to 5099
Batch 5100 to 5199
Batch 5200 to 5299
Batch 5300 to 53

## 4. Store document chunks and embeddings

Because this example only uses a few thousand strings, we'll store them in a CSV file.

(For larger datasets, use a vector database, which will be more performant.)

In [26]:
# save document chunks and embeddings

SAVE_PATH = "../data/processed/winter_olympics_2022.csv"

df.head(100).to_csv(SAVE_PATH, index=False)
